## Transfer Data: Transfer Data from Blob Storage to Azure SQL Database using an existing Azure Data Factory

### Description

[Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) 
Connect to your Azure Machine Learning service Workspace. Note that if you are using a Compute Instance the config file is already there for the Workspace you're within, otherwise you will need to create it.

In [1]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

Workspace Name: mlonazure-ws
Resource Group: mlonazure-rg
Default Storage Account Name: mlonazurews7669880942
AzureML Core Version: 1.5.0


### Description

[DataReference Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.data_reference.datareference?view=azure-ml-py) Create a reference to the blob storage. Note that in the path_on_datastore you can use wildcards for multipe files.


In [2]:
from azureml.data.data_reference import DataReference

blob_data_ref = DataReference(datastore = mlonazure_ds, 
                  data_reference_name='blobref', 
                  path_on_datastore='MyDatasets/OJSales_All/*.csv', 
                  mode='mount', 
                  path_on_compute=None, 
                  overwrite=False)

### Description

[SqlDataReference Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.sql_data_reference.sqldatareference?view=azure-ml-py) Create a reference to the SQL Datastore registered in (01. Transfer Data Configuration) and the specific table to write to. 

**Note** There is currently a bug in the SDK that writes to a table named 'dummy' and not the sql_table name passed here. Therefore, make sure you create the table 'dummy' (I know, I know...) and provide the Service Princple approperiate permissions to read/write to it.

**Note** The DataTransferStep assumes your data has a header and uses the first row to do a column mapping with your table's columns, the names need to match!


In [3]:
from azureml.data.sql_data_reference import SqlDataReference

mlonazuresql_ds = Datastore(workspace=mlonazure_ws,
                            name='azuresqldb_datastore')

sql_query_data_ref = SqlDataReference(
                    datastore = mlonazuresql_ds, 
                    data_reference_name='sqlreference', 
                    sql_table='OJSalesData', 
                    sql_query=None, 
                    sql_stored_procedure=None, 
                    sql_stored_procedure_params=None)


### Description

[DataFactoryCompute Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.datafactory.datafactorycompute?view=azure-ml-py) Simply connect to the datafactory that was attached in the 01. Transfer Data Coniguration Notebook.

In [4]:
from azureml.core.compute import DataFactoryCompute

adfcompute = 'mlonazure-adf'
adf_compute = DataFactoryCompute(workspace=mlonazure_ws, name=adfcompute)
adf_compute.name

'mlonazure-adf'

### Description

[DataTransferStep class] (https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.datatransferstep?view=azure-ml-py) Create a DataTransferStep providing it the blob reference, sql reference and the adf compute from above. 

**Note** allow_reuse is set to false so that the step always runs regardless if the configuration of the step is changed or not. If this is set to true then you might run your pipline and not see anything happen in ADF.

In [5]:
from azureml.pipeline.steps import DataTransferStep

datatransferstep_name = 'transfer_blob_to_sql'

data_transfer_step = DataTransferStep(name = datatransferstep_name, 
                         source_data_reference=blob_data_ref, 
                         destination_data_reference=sql_query_data_ref, 
                         compute_target=adf_compute, 
                         source_reference_type='file', 
                         #destination_reference_type=None, 
                         allow_reuse=False) #Allows reuse of prev results if settings are the same.

print("Data transfer step created")

Data transfer step created


### Description

[Pipeline class](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py) Create the pipeline with only one step which is the datatransferstep from above.

**Note** that the pipeline can have many steps.

In [6]:
from azureml.pipeline.core import Pipeline

datatransfer_pipeline = Pipeline(workspace=mlonazure_ws, 
         steps=[data_transfer_step], 
         description='Transfer blob data to sql')

### Description

[Experiment Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment%28class%29?view=azure-ml-py) Create an Experiment, check Studio to see your experiment is there now without any runs

Submit the experiment and wait for completion. While you wait for completion, go to Studio and click on Pipleines, you will see your pipeline run from there. 

**Note** Because this run is using ADF, you can go into the ADF you have and look at Monitor to see it. Errors may not come back up to Studio and therefore, see the errors in ADF

In [7]:
from azureml.core import Experiment 

exp = Experiment(workspace = mlonazure_ws, name="DataTransfer_BlobtoSQL")

exp_pipelinerun = exp.submit(datatransfer_pipeline)

exp_pipelinerun.wait_for_completion()

Created step transfer_blob_to_sql [bc408c37][101162a2-8e8c-4042-bc30-9f0dbc171965], (This step will run and generate new outputs)
Using data reference blobref for StepId [9f0c7820][36344740-5da1-4efa-84cc-7a6a9bc82307], (Consumers of this data are eligible to reuse prior runs.)
Using data reference sqlreference for StepId [f92ac1b2][8dbdc201-bc87-49c6-92fb-b54135317bf2], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 1facd3b9-e9a9-4013-9705-ad63370c0ec9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/DataTransfer_BlobtoSQL/runs/1facd3b9-e9a9-4013-9705-ad63370c0ec9?wsid=/subscriptions/286d2a8f-e90b-4e3f-b4d7-08971a2ab3fe/resourcegroups/mlonazure-rg/workspaces/mlonazure-ws
PipelineRunId: 1facd3b9-e9a9-4013-9705-ad63370c0ec9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/DataTransfer_BlobtoSQL/runs/1facd3b9-e9a9-4013-9705-ad63370c0ec9?wsid=/subscriptions/286d2a8f-e90b-4e3f-b4d7-08971a2ab3fe/resourcegroups/m

'Finished'

### Description

Publish the pipeline so that you have a Pipeline endpoint that can be called directly in the future. See Studio -> End Points -> Pipeline endpoints. Click on the endpoint and then you can submit there or you can use ADF or Azure Devops to call it. 

In [8]:
datatransfer_pipeline.publish(
        name='Transfer OJSales Data', 
        description='Transfer OJSales Data from Blob Storage to SQL Table', 
        version=None, 
        continue_on_step_failure=False)

Name,Id,Status,Endpoint
Transfer OJSales Data,3a82d7bb-ead5-4a33-a303-6e8995207840,Active,REST Endpoint
